In [20]:
import os
import geopandas as gpd
import numpy as np
from tqdm import tqdm

In [17]:
MERGED_DATA_PATH = '../Data/Speed Data/Merged' 
GRADIENT_DATA_PATH = '../Data/Gradient Data/Adjusted/'
OUTPUT_PATH = '../Data/Cycle Data/'

In [18]:
def closest(geom,possible):
    '''
    
    Input:
    geom - LineString geometry of bus route segment
    possible - LineString geometry of associated speed mesh segments found through spatial joining
    
    Output:
    i - index of possible which has the smallest angle with geom
    
    '''
    dy = geom.coords[1][1] - geom.coords[0][1]
    dx = geom.coords[1][0] - geom.coords[0][0]
    
    theta = np.arctan2(dy,dx)
    
    ans = 0
    close = 100000
    
    for i,e in enumerate(possible):
        dy = e.coords[1][1] - e.coords[0][1]
        dx = e.coords[1][0] - e.coords[0][0]
    
        t_theta = np.arctan2(dy,dx)
        
        if(abs(theta-t_theta) < close):
            close = abs(theta-t_theta)
            ans = i
    
    return i

In [35]:
speed_dfs = []


for month_bucket in range(4):
    speed_dfs.append([])
    
for k in tqdm(range(12),'Loading Speed Data'):
    month_bucket, hour_bucket = k//3, k%3
    path = os.path.join(MERGED_DATA_PATH, "%d_%d.shp" % (month_bucket, hour_bucket))
    df = gpd.read_file(path)
    df = df.to_crs('epsg:32614')
    
    df['non_buffered'] = df['geometry']
    df['geometry'] = df['geometry'].buffer(20)

    speed_dfs[month_bucket].append(df)

Loading Speed Data: 100%|████| 12/12 [4:00:15<00:00, 1201.27s/it]


In [31]:
shps = []
for file in os.listdir(GRADIENT_DATA_PATH):
    if ".shp" in file:
        shps.append(file)

for file in tqdm(shps, 'Cycle Generation', unit = 'route'):
    
    route = gpd.read_file(os.path.join(GRADIENT_DATA_PATH,file))
    route.crs = 'epsg:32614'
    
    for month_bucket in range(4):
        for hour_bucket in range(3):
            
            df = speed_dfs[month_bucket][hour_bucket]
            
            bucket = "%d_%d" % (month_bucket, hour_bucket)
            
            out_path = os.path.join(OUTPUT_PATH, bucket, file)
            
            result = gpd.sjoin(route,df)
            speeds = np.zeros(len(route['geometry']))
            
            for i in range(len(result.index)):
                try:
                    cur = list(result['index_right'][result.index[i]])
                except:
                    pass
            
                ind = closest(route['geometry'][result.index[i]],list(df['non_buffered'][cur]))
                poss = list(df['speed'][cur])
            
                speeds[result.index[i]] = poss[ind]
            
            route['speed'] = speeds
        
        

Cycle Generation:   0%|               | 0/109 [00:00<?, ?route/s]C:\Users\rahej\.conda\envs\machinelearning\lib\site-packages\geopandas\tools\sjoin.py:61: UserWarning: CRS of frames being joined does not match!(epsg:32614 != epsg:4326)
  "(%s != %s)" % (left_df.crs, right_df.crs)


KeyboardInterrupt: 

Cycle Generation:   0%|               | 0/109 [00:07<?, ?route/s]


NameError: name 'i' is not defined

In [ ]:
df = gpd.read_file('Merged/0_0.shp')
df = df.to_crs('epsg:32614')

df['non_buffered'] = df['geometry']
df['geometry'] = df['geometry'].buffer(0.002)

for dirname, _, filenames in os.walk('./Gradient'):
    for filename in filenames:
        
        file = os.path.join(dirname, filename)
        if ".shp" not in file:
            continue
        name = filename.replace(".shp","")

        print('COMPUTING CYCLE FOR ',name)

        route = gpd.read_file(file)
        route.crs = 'epsg:32614'

        print("MERGING")
        result = gpd.sjoin(route,df)
        print("DONE MERGING")

        print("ASSINGNING SPEEDS")
        speeds = np.zeros(len(result['geometry']))

        for i in range(len(result.index)):
            try:
                cur = list(result['index_right'][result.index[i]])
            except:
                pass
            
            ind = closest(route['geometry'][result.index[i]],list(df['non_buffered'][cur]))
            poss = list(df['speed'][cur])
            
            speeds[result.index[i]] = poss[ind]
        print("DONE ASSINGNING SPEEDS")

        route['speed'] = speeds[0:len(route)]

        speeds = []
        grades = []

        time_left = 1
        cur_speed = 0
        cur_grade = 0
        empty = 0

        for i in range(len(route['geometry'])):
            geom = route['geometry'][i]
            speed = route['speed'][i] 
            if speed == 0:
                empty += 1
                speed = 12.5
            grade = route['grade'][i]
            len_left = geom.length
            while(len_left > 0):
                if(len_left > speed*time_left):
                    len_left -= speed*time_left
                    cur_speed += time_left*speed
                    cur_grade += time_left*grade
                    grades.append(cur_grade)
                    speeds.append(cur_speed)
                    cur_speed = 0
                    cur_grade = 0
                    time_left = 1
                else:
                    time_spent = len_left/speed 
                    len_left = 0
                    time_left -= time_spent
                    cur_speed += time_spent*speed
                    cur_grade += time_spent*grade


        print(empty/len(route['geometry']),'points with no data')
        f = open('Cycles/' + name + ".csv","w")
        f.write("Time (seconds),Speed,Grade,Road Type\n")
        for i in range(len(grades)):
            f.write(str(i) + "," + str(speeds[i]) + "," + str(grades[i]) +",No Charging\n")
        f.close()

In [34]:
df

,speed,num,geometry,non_buffered
0,1.755556,118,"POLYGON ((-119.64687 18.80003, -119.59863 20.7...","LINESTRING (-99.65283 19.28813, -99.65288 19.2..."
1,1.111111,59,"POLYGON ((-99.69184 39.29075, -97.73130 39.198...","LINESTRING (-99.65155 19.29079, -99.65011 19.2..."
2,1.477778,59,"POLYGON ((-79.65070 19.58815, -79.71886 17.626...","LINESTRING (-99.64867 19.30298, -99.64864 19.3..."
3,0.880556,59,"POLYGON ((-100.03518 39.29964, -98.07333 39.24...","LINESTRING (-99.64782 19.30339, -99.64659 19.3..."
4,1.119444,59,"POLYGON ((-79.64501 19.43665, -79.72703 17.475...","LINESTRING (-99.64449 19.29225, -99.64448 19.2..."
...,...,...,...,...
76458,16.272222,59,"POLYGON ((-107.19500 37.44363, -107.13175 37.4...","LINESTRING (-98.71900 19.33225, -98.71915 19.3..."
76459,17.175000,236,"POLYGON ((-107.17480 37.45423, -107.13175 37.4...","LINESTRING (-98.71900 19.33225, -98.71915 19.3..."
76460,13.837500,118,"POLYGON ((-117.32517 11.82229, -117.50653 12.2...","LINESTRING (-98.80079 19.31645, -98.79727 19.3..."
76461,13.272222,59,"POLYGON ((-117.32372 11.82288, -117.50329 12.2...","LINESTRING (-98.79909 19.31673, -98.79727 19.3..."
